In [52]:
!pip install -q zeugma

In [53]:
import pandas as pd
import numpy as np
import nltk

from sklearn.metrics import roc_auc_score,accuracy_score

In [54]:
df = pd.read_csv('reviews.txt',sep = '\t', names =['Reviews','Comments'])
df

,Reviews,Comments
0,1,The Da Vinci Code book is just awesome.
1,1,this was the first clive cussler i've ever rea...
2,1,i liked the Da Vinci Code a lot.
3,1,i liked the Da Vinci Code a lot.
4,1,I liked the Da Vinci Code but it ultimatly did...
...,...,...
6913,0,Brokeback Mountain was boring.
6914,0,So Brokeback Mountain was really depressing.
6915,0,"As I sit here, watching the MTV Movie Awards, ..."
6916,0,Ok brokeback mountain is such a horrible movie.


In [55]:
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer


nltk.download('omw-1.4',download_dir='.')
nltk.download('wordnet',download_dir='.')
nltk.download('punkt', download_dir='.')
nltk.data.path.append('.')

# lowercase
# Remove non-sense(digits, url, ..)
# Stripping
# Punctuation
# Remove Stopword
# Steeming/Lemmatization

steemer = PorterStemmer()
lemmatizater = WordNetLemmatizer()
def stem(x):
    L = []
    for i in x.split():
        L.append(steemer.stem(i))

    return ' '.join(L)

def text_preprocessing(x):
    x = x.lower()
    x = re.compile(r'https?://\S+').sub('', x)
    x = re.sub(r'[^\w\s]', '', x)
    x = re.sub(r'\d', '', x)
    tokens = word_tokenize(x)
    tokens = [steemer.stem(i) for i in tokens]
    tokens = [lemmatizater.lemmatize(i) for i in tokens]

    return ' '.join(tokens)

df['Clean'] = df['Comments'].apply(text_preprocessing)

[nltk_data] Downloading package omw-1.4 to ....
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to ....
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to ....
[nltk_data]   Package punkt is already up-to-date!


In [56]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from zeugma.embeddings import EmbeddingTransformer

# vectorizer=CountVectorizer(max_features=2000, strip_accents='ascii',stop_words='english')
# vectorizer = TfidfVectorizer(use_idf = True, lowercase = True, strip_accents='ascii')
vectorizer = EmbeddingTransformer('glove')

[==================================================] 100.0% 104.8/104.8MB downloaded


In [57]:
X = vectorizer.fit_transform(df.Comments)
y = df.Reviews

In [58]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [60]:
from sklearn.ensemble import RandomForestClassifier,ExtraTreesClassifier

clf=RandomForestClassifier()
clf.fit(X_train,y_train)
accuracy_score(y_test,clf.predict(X_test))*100

92.34104046242774

In [61]:
clf = RandomForestClassifier()
clf.fit(X,y)
accuracy_score(y_test,clf.predict(X_test))*100


95.59248554913295

In [62]:
import pickle

pickle.dump(clf, open('sentiment_cls.pkl', 'wb'))
pickle.dump(vectorizer, open('vectorizer.pkl', 'wb'))